In [1]:
import keras
import numpy as np
import os
import random
import tensorflow as tf
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
keras.__version__

c:\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.4'

In [2]:
# Get images
X = []
for filename in os.listdir('../Full-version/Train/'):
    X.append(img_to_array(load_img('../Full-version/Train/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [52]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
#model.compile(optimizer='rmsprop', loss='mse',metrics=['acc'])
#model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='mse',metrics=['acc'])
#model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='mse',metrics=['acc'])

#optz = optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
#optz = optimizers.RMSprop(lr=0.00001)
optz = optimizers.Adam(lr=0.00001)
#optz = 'categorical_crossentropy'
#optz='rmsprop'
model.compile(optimizer=optz, loss='mse',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_206 (Conv2D)          (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_207 (Conv2D)          (None, 128, 128, 64)      36928     
_________________________________________________________________
conv2d_208 (Conv2D)          (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_209 (Conv2D)          (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_210 (Conv2D)          (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_211 (Conv2D)          (None, 32, 32, 256)       590080    
__________

In [57]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 5
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
history = model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=100, steps_per_epoch=200)

Epoch 1/100
200/200 [==============================] - 70s 350ms/step - loss: 0.0102 - acc: 0.5480
Epoch 2/100
200/200 [==============================] - 69s 346ms/step - loss: 0.0100 - acc: 0.5519
Epoch 3/100
200/200 [==============================] - 69s 346ms/step - loss: 0.0100 - acc: 0.5542
Epoch 4/100
200/200 [==============================] - 69s 346ms/step - loss: 0.0099 - acc: 0.5591
Epoch 5/100
200/200 [==============================] - 69s 347ms/step - loss: 0.0100 - acc: 0.5545
Epoch 6/100
200/200 [==============================] - 69s 347ms/step - loss: 0.0100 - acc: 0.5565
Epoch 7/100
200/200 [==============================] - 69s 347ms/step - loss: 0.0094 - acc: 0.5594
Epoch 8/100
200/200 [==============================] - 69s 347ms/step - loss: 0.0102 - acc: 0.5620
Epoch 9/100
200/200 [==============================] - 69s 347ms/step - loss: 0.0100 - acc: 0.5583
Epoch 10/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0097 - acc: 0.5546
Epoch 11/

200/200 [==============================] - 69s 345ms/step - loss: 0.0093 - acc: 0.5850
Epoch 84/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0092 - acc: 0.5719
Epoch 85/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0093 - acc: 0.5811
Epoch 86/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0094 - acc: 0.5768
Epoch 87/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0095 - acc: 0.5820
Epoch 88/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0092 - acc: 0.5835
Epoch 89/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0093 - acc: 0.5846
Epoch 90/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0089 - acc: 0.5758
Epoch 91/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0095 - acc: 0.5826
Epoch 92/100
200/200 [==============================] - 69s 345ms/step - loss: 0.0089 - acc: 0.5873
Epoch 93/100


In [54]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [55]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

50/50 [==============================] - 1s 25ms/step
[0.014036951493471861, 0.5152743458747864]


In [56]:
color_me = []
for filename in os.listdir('../Full-version/Test/'):
    color_me.append(img_to_array(load_img('../Full-version/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\anaconda3\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
